In [15]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
import os, pathlib
FILE_DIR = os.getcwd()

# Fuel Types
FUEL = ['coal', 'NGCC', 'nuclear', 'hydro', 'wind', 'solar', 'ev']

# Carbon Emission in gCO2e per kWh for each fuel type
E = np.array([820, 490, 12, 24, 11.5, 44.5, 0,])

# LCOE in $/kWh for each fuel type from AEO2023
C = np.array([89.33, 42.72, 71.00, 57.12, 31.07, 23.22, 36.27])/1e3

# Number of fuel types
NUMFT = len(E)

# State level data on population, energy demand, current co2 emissions from electricity
FNAME = os.path.join(FILE_DIR, "StateData.xlsx")
STATES = pd.read_excel(FNAME, sheet_name="StateData", index_col=0, header=0)

# Assume some installed generation capacity for the state (in kW)
# In a more advanced model, these could be decision variables.
# For WA, let's assume a hydro-heavy mix with growing renewables.
INSTALLED_CAPACITY_KW = {
    'coal': 200000000, # Washington has phased out coal
    'NGCC': 5_000_000,
    'nuclear': 1_200_000,
    'hydro': 20_000_000,
    'wind': 4_000_000,
    'solar': 3_000_000,
    'ev': 0
}
INSTALLED_CAPACITY_KW_array = np.array([INSTALLED_CAPACITY_KW[f] for f in FUEL])

In [17]:
state = 'WA'
futureDemandGrowth = 1.0 # Future electricity demand as pct of current
TotalDemand = STATES['Demand (MWh)'][state]*1e3*futureDemandGrowth # convert to kwh
futureCo2Improve = 0.999 # Future emissions target as pct of current
LimitCO2 = STATES['CO2 Emissions (million metric tons)'][state]*1e12*futureCo2Improve #convert to gCO2e

In [18]:
### DATA PREPARATION FOR HOURLY MODEL ###

# Number of time steps in our model (24 hours)
T = 24
hours = np.arange(T)

# 1. Simulate the Duck Curve (Net Demand) for a representative day
# This curve shows high demand in the morning and a sharp ramp-up in the evening,
# with a "belly" in the middle of the day due to high solar generation.
# We normalize it first, then scale it by the state's total demand.
duck_curve_normalized = np.array([
    0.60, 0.58, 0.57, 0.56, 0.57, 0.61, 0.70, 0.75, # Morning
    0.65, 0.55, 0.45, 0.38, 0.35, 0.33, 0.35, 0.45, # Midday "Belly"
    0.60, 0.80, 0.95, 1.00, 0.98, 0.90, 0.80, 0.70  # Evening Peak
])
daily_demand_mwh = STATES['Demand (MWh)'][state] * futureDemandGrowth / 365
HRLYDEMAND = duck_curve_normalized * daily_demand_mwh * 1000 # Convert to kWh

# 2. Define Hourly Capacity Factors (AEO2023)
# This is crucial. Solar only works during the day. Wind might be stronger at night.
# Dispatchable generation (Coal, NGCC, Nuclear) is assumed to be available 24/7.
# We create a DataFrame for clarity.
HRLY_CAP_FACTORS = pd.DataFrame(index=FUEL, columns=hours, data=1.0) # Default to 100% available
HRLY_CAP_FACTORS.loc['solar'] = [0, 0, 0, 0, 0, 0.1, 0.3, 0.5, 0.7, 0.85, 0.95, 1.0, 1.0, 0.95, 0.85, 0.7, 0.5, 0.2, 0, 0, 0, 0, 0, 0]
HRLY_CAP_FACTORS.loc['wind']  = [0.5, 0.52, 0.53, 0.51, 0.48, 0.45, 0.42, 0.43, 0.45, 0.4, 0.35, 0.3, 0.28, 0.3, 0.32, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.58, 0.55, 0.52]
# Set EV to 0, as it's a storage/demand source, not generation
HRLY_CAP_FACTORS.loc['ev'] = 0.0

In [19]:
# 3. Define Battery Bank Parameters
# Let's assume a large battery bank, e.g., 20% of the daily peak demand capacity

# TODO Incorporate 15087 results in here
# EV Capacity (avg) in kWh
EVC_MAX = 80
# Daily EV energy usage in kWh
EVC_MIN = 5*11106/3.5/365 # safety factor * avg annual miles / avg miles/kWh / days per year

BATTERY_MAX_ENERGY_KWH = np.max(HRLYDEMAND) * 4  # Capacity to store 4 hours of peak demand
BATTERY_MAX_POWER_KW = BATTERY_MAX_ENERGY_KWH / 4 # Max charge/discharge rate (C/4 rate)
BATTERY_CHARGE_EFFICIENCY = 0.95 # 95% efficiency when charging
BATTERY_INITIAL_SOC_KWH = BATTERY_MAX_ENERGY_KWH / 2 # Start the day at 50% charge

In [ ]:
### --- REVISED OPTIMIZATION MODEL --- ###

# Initialize the model
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env, name="Hourly_Energy_Dispatch_with_Battery")

### DECISION VARIABLES ###

# generation[i, t]: kWh generated by fuel i in hour t
generation = m.addMVar((NUMFT, T), vtype=GRB.CONTINUOUS, name="generation")

# charge[t]: kWh sent TO the battery in hour t
charge = m.addMVar(T, vtype=GRB.CONTINUOUS, name="charge")

# discharge[t]: kWh sent FROM the battery in hour t
discharge = m.addMVar(T, vtype=GRB.CONTINUOUS, name="discharge")

# soc[t]: State of Charge (energy in kWh) of the battery at the END of hour t
soc = m.addMVar(T, vtype=GRB.CONTINUOUS, name="soc")


### OBJECTIVE FUNCTION ###
# Minimize the total cost of generation over the 24-hour period
m.setObjective(gp.quicksum(generation[i, t] * C[i] for i in range(NUMFT) for t in range(T)), GRB.MINIMIZE)


### CONSTRAINTS ###

# 1. Hourly Demand Fulfillment Constraint
# For each hour, total generation + battery discharge - battery charge must equal demand.
m.addConstrs((generation.sum(axis=0)[t] + discharge[t] - charge[t] == HRLYDEMAND[t] for t in range(T)), name="DemandFulfillment")

# 2. Generation Capacity Constraint
# Each generator's output in an hour cannot exceed its installed capacity * its hourly capacity factor.
m.addConstrs((generation[i, t] <= INSTALLED_CAPACITY_KW_array[i] * HRLY_CAP_FACTORS.loc[FUEL[i], t] for i in range(NUMFT) for t in range(T)), name="GenCapacity")

# 3. Total CO2 Emissions Constraint
# Sum of all emissions over 24 hours must be below the daily limit.
daily_co2_limit = LimitCO2 / 365
m.addConstr(gp.quicksum(generation[i, t] * E[i] for i in range(NUMFT) for t in range(T)) <= daily_co2_limit, name="CO2_Limit")

# --- BATTERY CONSTRAINTS ---

# 4. Battery State of Charge (SoC) Evolution
# This is the key constraint linking time periods.
# SoC at end of hour t = SoC at end of hour t-1 + energy charged - energy discharged.
m.addConstr(soc[0] == BATTERY_INITIAL_SOC_KWH + charge[0] * BATTERY_CHARGE_EFFICIENCY - discharge[0], name="SoC_Initial")
m.addConstrs((soc[t] == soc[t-1] + charge[t] * BATTERY_CHARGE_EFFICIENCY - discharge[t] for t in range(1, T)), name="SoC_Evolution")

# 5. Battery Power Limits
# The rate of charge/discharge is limited.
m.addConstrs((charge[t] <= BATTERY_MAX_POWER_KW for t in range(T)), name="MaxChargeRate")
m.addConstrs((discharge[t] <= BATTERY_MAX_POWER_KW for t in range(T)), name="MaxDischargeRate")

# 6. Battery Energy Limits
# The stored energy cannot exceed the battery's maximum capacity.
m.addConstrs((soc[t] <= BATTERY_MAX_ENERGY_KWH for t in range(T)), name="MaxSoC")
m.addConstrs((soc[t] >= 0 for t in range(T)), name="MinSoC") # Can't have negative energy

# 7. (Optional but good practice) Cyclic Constraint
# Ensure the battery has at least as much energy at the end of the day as it started with.
# m.addConstr(soc[T-1] >= BATTERY_INITIAL_SOC_KWH, name="CyclicSoC")

### SOLVE THE MODEL ###
m.optimize()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 313 rows, 240 columns and 719 nonzeros
Model fingerprint: 0xf0bef833
Coefficient statistics:
  Matrix range     [9e-01, 8e+02]
  Objective range  [2e-02, 9e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+05, 3e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 266 rows and 37 columns
Presolve time: 0.00s
Presolved: 47 rows, 203 columns, 405 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.969060e+08   0.000000e+00      0s

Solved in 29 iterations and 0.00 seconds (0.00 work units)
Infeasible model


In [21]:
### VISUALIZE THE RESULTS ###

if m.Status == GRB.OPTIMAL:
    print(f"Optimal daily cost: ${m.ObjVal:,.2f}")

    # Extract solution data
    gen_sol = pd.DataFrame(generation.X, index=FUEL, columns=hours)
    charge_sol = charge.X
    discharge_sol = discharge.X
    soc_sol = soc.X
    
    # Create the plot
    fig, ax1 = plt.subplots(figsize=(15, 8))
    
    # Stacked bar chart for generation
    bottom = np.zeros(T)
    colors = plt.cm.viridis(np.linspace(0, 1, NUMFT))
    for i, fuel in enumerate(FUEL):
        if fuel != 'ev': # Don't plot EV generation
            ax1.bar(hours, gen_sol.loc[fuel], bottom=bottom, label=fuel.capitalize(), color=colors[i], width=1.0)
            bottom += gen_sol.loc[fuel]
            
    # Plot net demand (the duck curve)
    ax1.plot(hours, HRLYDEMAND, 'r--', linewidth=3, label='Hourly Demand (Duck Curve)')
    
    # Plot battery discharge (adds to supply)
    ax1.bar(hours, discharge_sol, bottom=bottom, label='Battery Discharge', color='gold', width=1.0)

    ax1.set_xlabel('Hour of the Day')
    ax1.set_ylabel('Energy (kWh)')
    ax1.set_title(f'Optimal Hourly Energy Dispatch for {state} with Battery Storage')
    ax1.legend(loc='upper left')
    ax1.grid(True)
    ax1.set_xticks(hours)
    ax1.set_xlim(-0.5, 23.5)

    # Create a second y-axis for the battery State of Charge
    ax2 = ax1.twinx()
    # Plot battery charge (negative value to show it's a load)
    ax2.plot(hours, -charge_sol, 'm-.', linewidth=2, label='Battery Charge')
    # Plot State of Charge
    ax2.plot(hours, soc_sol, 'b-', linewidth=3, label='Battery State of Charge (kWh)')
    ax2.set_ylabel('Battery State of Charge (kWh) / Charge Rate (kWh)', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2.legend(loc='upper right')
    
    fig.tight_layout()
    plt.show()

else:
    print("Optimization was not successful. Status:", m.Status)

Optimization was not successful. Status: 3
